In [20]:
# from ../../Physionet_rev

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from src import definitions
import pprint
pp = pprint.PrettyPrinter(indent=4)
from IPython.display import display
from src.utils import read_utils as reader
from src.utils.read_utils import read_yaml
from src.utils import student_utils
from src.data_getter import student_life_var_binned_data_getter as sl_getter
from src.definitions import MINIMAL_PROCESSED_DATA_PATH, \
    FEATURE_CONFIG_FILE_PATH, STUDENT_FOLDER_NAME_PREFIX, BINNED_ON_VAR_FREQ_DATA_PATH
from src.data_processing import helper

import sklearn
from sklearn import cluster as CL
from sklearn.decomposition import PCA


In [2]:
data_path = module_path + "/data"
print(data_path + " -- " + str(os.path.exists(data_path)))
stress_csv_data_path = data_path + "/stress_distribution.csv"
print(stress_csv_data_path + " -- " + str(os.path.exists(stress_csv_data_path)))

/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data -- True
/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data/stress_distribution.csv -- True


In [3]:
data = None
with open(stress_csv_data_path, 'rb') as data_file:
    data = pd.read_csv(data_file)
print(list(data))

['student_id', 'stress_level_stress_0', 'stress_level_stress_1', 'stress_level_stress_2', 'stress_level_stress_3', 'stress_level_stress_4', 'cluster_', 'svg_stress_score_']


In [4]:
X = data[['stress_level_stress_0', 'stress_level_stress_1',
             'stress_level_stress_2', 'stress_level_stress_3',
             'stress_level_stress_4', 'svg_stress_score_']]
print(X.iloc[0])
print(len(X))

stress_level_stress_0      6
stress_level_stress_1     27
stress_level_stress_2     48
stress_level_stress_3      9
stress_level_stress_4     18
svg_stress_score_        330
Name: 0, dtype: int64
46


In [17]:
K = 5
def cluster(estimator, data):
    reduced_data = PCA(n_components=2).fit_transform(data)
    print(estimator)
    print()

    estimator.fit(X)
    centroids = estimator.cluster_centers_
    labels = estimator.labels_
    
    return centroids, labels

In [19]:
K = 5
cen, clus = cluster(CL.KMeans(init="k-means++", n_clusters=K, n_init=K), data)
print(cen)
print(clus)
X['hist_cluster'] = list(clus)
print(X.iloc[0])y

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=5, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

[[ 7.75000000e+00  2.80000000e+01  6.72500000e+01  3.12500000e+01
   3.55000000e+01  5.68000000e+02 -2.22044605e-16]
 [ 4.60000000e+00  1.04000000e+01  2.36000000e+01  7.20000000e+00
   7.20000000e+00  1.61000000e+02  1.00000000e+00]
 [ 6.00000000e+01  2.67000000e+02  3.99000000e+02  2.10000000e+01
   6.00000000e+01  2.17500000e+03  2.00000000e+00]
 [ 4.50000000e+00  4.85000000e+01  1.35000000e+02  4.80000000e+01
   5.85000000e+01  9.91000000e+02  3.00000000e+00]
 [ 8.50000000e+00  2.80000000e+01  5.57500000e+01  1.40000000e+01
   1.60000000e+01  3.67750000e+02  4.00000000e+00]]
[4 4 0 1 4 3 1 3 4 0 1 3 0 1 3 1 0 4 0 1 4 4 1 0 0 1 4 0 1 1 4 3 3 1 4 1 0
 1 0 4 4 1 1 0 0 2]
stress_level_stress_0      6
stress_level_stress_1     27
stress_level_stress_2     48
stress_level_stress_3      9
stress_level_str

/Users/nsimsiri/anaconda2/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [34]:
d = {}
for i, x in enumerate(clus):
    if x in d: d[x]+=[i]
    else: d[x] = [i]
print(d)
import collections
collections.OrderedDict(sorted(d.items()))

{4: [0, 1, 4, 8, 17, 20, 21, 26, 30, 34, 39, 40], 0: [2, 9, 12, 16, 18, 23, 24, 27, 36, 38, 43, 44], 1: [3, 6, 10, 13, 15, 19, 22, 25, 28, 29, 33, 35, 37, 41, 42], 3: [5, 7, 11, 14, 31, 32], 2: [45]}


OrderedDict([(0, [2, 9, 12, 16, 18, 23, 24, 27, 36, 38, 43, 44]),
             (1, [3, 6, 10, 13, 15, 19, 22, 25, 28, 29, 33, 35, 37, 41, 42]),
             (2, [45]),
             (3, [5, 7, 11, 14, 31, 32]),
             (4, [0, 1, 4, 8, 17, 20, 21, 26, 30, 34, 39, 40])])

In [37]:
ori = list(data['cluster_'])
# print(ori)
print(ori)
d = {}
for i, x in enumerate(ori):
    if x in d: d[x]+=[i]
    else: d[x] = [i]
print(d)
import collections
pp.pprint(collections.OrderedDict(sorted(d.items())))

[2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 1]
{2: [0, 1, 2, 3, 4, 6, 8, 9, 10, 12, 13, 15, 18, 19, 20, 21, 22, 24, 25, 26, 28, 29, 30, 33, 34, 35, 36, 37, 39, 41, 42], 0: [5, 7, 11, 14, 16, 17, 23, 27, 31, 32, 38, 40, 43, 44], 1: [45]}
OrderedDict([   (0, [5, 7, 11, 14, 16, 17, 23, 27, 31, 32, 38, 40, 43, 44]),
                (1, [45]),
                (   2,
                    [   0,
                        1,
                        2,
                        3,
                        4,
                        6,
                        8,
                        9,
                        10,
                        12,
                        13,
                        15,
                        18,
                        19,
                        20,
                        21,
                        22,
                        24,
                        25,
                    

In [ ]:
# OrderedDict([(0, 14), (1, 1), (2, 31)])
# OrderedDict([(0, 12), (1, 15), (2, 1), (3, 6), (4, 12)])